In [9]:
from PIL import Image # هذه المكتبة تتعلق بمعالجة الصور
from sklearn.model_selection import train_test_split  # تقسيم البيانات
import tensorflow as tf  # مكتبة تعلم عميق
import numpy as np  # للعمل مع المصفوفات
import matplotlib.pyplot as plt  # للرسم البياني وعرض الصور
import os  # للعمل مع نظام الملفات
import time  # حساب الوقت
from tensorflow import keras  # مكتبة تعلم عميق فرعية

# تشغيل الوظائف بوضع التنفيذ الفوري (للإصدارات الأحدث من TensorFlow)
tf.config.run_functions_eagerly(True)

# حجم الدفعة الذي سنستخدمه أثناء التدريب
batch_size = 64

# حجم الصورة المطلوب لتدريب النموذج
img_size = 120

# عدد الصور التي سيتم استخدامها من الأرشيف
dataset_split = 2500

master_dir = r'D:\1-ForME\Project\AI for AMIT\colorization\Dataset\archive\color'
x = []
y = []
for image_file in os.listdir(master_dir)[0:dataset_split]:
    rgb_image = Image.open(os.path.join(master_dir, image_file)).resize((img_size, img_size))
    # تطبيع صورة RGB
    rgb_img_array = (np.asarray(rgb_image)) / 255
    gray_image = rgb_image.convert('L')
    # تطبيع صورة تدرج الرمادي
    gray_img_array = (np.asarray(gray_image).reshape((img_size, img_size, 1))) / 255
    # إضافة الصور إلى القوائم
    x.append(gray_img_array)
    y.append(rgb_img_array)

# تقسيم البيانات إلى تدريب واختبار
train_x, test_x, train_y, test_y = train_test_split(np.array(x), np.array(y), test_size=0.1)

# إنشاء كائن tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(batch_size)


ModuleNotFoundError: No module named 'PIL'

## **2. The GAN**

In this section, we'll create our GAN model step-by-step with Keras. First, we'll implement the generator then the discriminator and finally the loss functions required by both of them.


### **A. Generator** 

Our generator ( represented as $G$ ) will take in grayscale image $x$ and produce a RGB image $G( x )$. Note, $x$ will be a tensor of shape $( \ batch \ size \ , \ 120 \ , \ 120 \ , \ 1 \ )$ and the output $G(x)$ will have a shape $( \ batch \ size \ , \ 120 \ , \ 120 \ , \ 3 \ )$

* Our generator will have a encoder-decoder structure, similar to the UNet architecture. Additionally, we use Dilated convolutions to have a larger receptive field.

* We introduce skip connections in our model so as to have better flow of information from the encoder to the decoder.
